In [ ]:
# WARNING: advised to install a specific version, e.g. tensorwaves==0.1.2
%pip install -q tensorwaves[doc,jax,pwa,viz] IPython

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Amplitude analysis

While TensorWaves can handle [general mathematical expressions](./usage.ipynb), it was originally created to perform **amplitude analysis** / **Partial Wave Analysis** (PWA), that is, to fit amplitude models to four-momenta data samples.

This notebook shows how to do an amplitude analysis with the [ComPWA](https://github.com/ComPWA) packages [QRules](https://qrules.rtfd.io), [AmpForm](https://AmpForm.rtfd.io), and [TensorWaves](https://tensorwaves.rtfd.io). The ComPWA workflow generally consists of three stages:

1. [Create an amplitude model](compwa-step-1) with {mod}`qrules` and {mod}`ampform`.
2. [Generate hit-and-miss data samples](compwa-step-2) with this amplitude model.
3. [Fit model to the data samples](compwa-step-3).

:::{note}

This notebook shows several tricks that _can_ be helpful when doing an amplitude analysis. **Most steps are optional** though—they only serve to illustrate some tips that can be adopted and worked out further for specific analyses.

:::

In [ ]:
from __future__ import annotations

(compwa-step-1)=
## Step 1: Formulate model

Whether [generating data](compwa-step-2) or [fitting a model](compwa-step-3), TensorWaves takes mathematical expressions as input. When that expression is an amplitude model, it is most convenient to formulate it with {mod}`qrules` and {mod}`ampform`.

### 1.1 Generate transitions

The first step is to generate all allowed transitions with {doc}`QRules <qrules:index>`. In this example, we use the helicity formalism, but you can also use `formalism="canonical-helicity"`. As you can see, we analyze the decay $J/\psi \to \gamma\pi^0\pi^0$ here.

:::{admonition} Simplified model: $J/\psi \to \gamma f_0$, $f_0 \rightarrow \pi^0\pi^0$
---
class: dropdown
---

As [](compwa-step-3) serves to illustrate usage only, we make the amplitude model here a bit simpler by not allowing $\omega$ resonances (which are narrow and therefore hard to fit). For this reason, we can also limit the {class}`~qrules.settings.InteractionType` to {attr}`~qrules.settings.InteractionType.STRONG`.

:::

In [ ]:
import qrules

reaction = qrules.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)"],
    allowed_interaction_types=["strong", "EM"],
    formalism="helicity",
)

In [ ]:
import graphviz

dot = qrules.io.asdot(reaction, collapse_graphs=True)
graphviz.Source(dot)

:::{tip}

See more advanced examples on {doc}`QRules' usage page <qrules:usage>`.

:::

(build-amplitude-model)=
### 1.2 Build amplitude model

Next, we use {mod}`ampform` to formulate the {attr}`~qrules.transition.ReactionInfo.transitions` as an amplitude model (here: {class}`~ampform.helicity.HelicityModel`). This can be done with {func}`~ampform.get_builder` and {meth}`~ampform.helicity.HelicityAmplitudeBuilder.formulate`:

In [ ]:
import ampform

model_builder = ampform.get_builder(reaction)
model_no_dynamics = model_builder.formulate()
model_no_dynamics.intensity

In [ ]:
from ampform.io import aslatex
from IPython.display import Math

Math(aslatex(model_no_dynamics.amplitudes))

In [ ]:
Math(aslatex(model_no_dynamics.parameter_defaults))

The heart of the model is a sympy {attr}`~ampform.helicity.HelicityModel.expression` that contains the full description of the intensity model. Note two things:
1. The coefficients for the different amplitudes are **complex** valued.
2. By default there is no dynamics in the model, so it still has to be specified.

We choose to use {func}`~ampform.dynamics.relativistic_breit_wigner_with_ff` as the lineshape for all resonances and use a Blatt-Weisskopf form factor ({func}`~ampform.dynamics.builder.create_non_dynamic_with_ff`) for the production decay. The {meth}`~ampform.helicity.DynamicsSelector.assign` method of the {attr}`~ampform.helicity.HelicityAmplitudeBuilder.dynamics` attribute is a convenience interface for replacing the dynamics for intermediate states.

In [ ]:
from ampform.dynamics.builder import (
    create_non_dynamic_with_ff,
    create_relativistic_breit_wigner_with_ff,
)

model_builder.dynamics.assign("J/psi(1S)", create_non_dynamic_with_ff)
for name in reaction.get_intermediate_particles().names:
    model_builder.dynamics.assign(name, create_relativistic_breit_wigner_with_ff)
model = model_builder.formulate()

Now let's take another look at the parameters of the model to see which new parameters are there:

In [ ]:
sorted_parameter_defaults = {
    symbol: model.parameter_defaults[symbol]
    for symbol in sorted(model.parameter_defaults, key=str)
}
src = aslatex(sorted_parameter_defaults)
Math(src)

Optionally, we can backup the {class}`~ampform.helicity.HelicityModel` to disk via {mod}`pickle`. The {class}`~qrules.transition.ReactionInfo` object (which takes longest to generate) can also be pickled, or it can also be serialized to JSON:

In [ ]:
import pickle

qrules.io.write(reaction, "transitions.json")
with open("helicity_model.pickle", "wb") as stream:
    pickle.dump(model, stream)

In the next steps, we use this {class}`~ampform.helicity.HelicityModel` as a template for a computational function to [generate data](compwa-step-2) and to [perform a fit](compwa-step-3).

:::{tip}

See more advanced examples on {doc}`AmpForm's usage page <ampform:usage>`.

:::

(compwa-step-2)=
## Step 2: Generate data

::::{margin}
:::{tip}
{doc}`TR-018<compwa-report:018/index>` explains some of the mechanisms behind the phase space generator as well as how to do {ref}`importance sampling<compwa-report:018/index:Intensity distribution>`.
:::
::::

In this section, we use the {class}`~ampform.helicity.HelicityModel` that we created with {mod}`ampform` in [the previous step](compwa-step-1) to generate a data sample via hit & miss Monte Carlo. We do this with the {mod}`.data` module.

**Optionally**, we can {func}`~pickle.load` the {class}`~ampform.helicity.HelicityModel` that was created in the previous step. This does not have to be done if the model has been generated in the same script or notebook (like in this notebook), but can be useful if the model was generated elsewhere.

In [ ]:
import pickle

from ampform.helicity import HelicityModel

with open("helicity_model.pickle", "rb") as model_file:
    imported_model: HelicityModel = pickle.load(model_file)

In [ ]:
initial_state, *_ = imported_model.reaction_info.initial_state.values()
print("Initial state:")
print(" ", initial_state.name)
print("Final state:")
for i, p in imported_model.reaction_info.final_state.items():
    print(f"  {i}: {p.name}")
del initial_state

(compwa-step-2.1)=
### 2.1 Generate phase space sample

The {class}`~qrules.transition.ReactionInfo` class defines the constraints of the phase space. As such, we have enough information to generate a **phase-space sample** for this particle reaction. We do this with a {class}`.TFPhaseSpaceGenerator` class, which is a {class}`.DataGenerator` for a {obj}`.DataSample` of **four-momenta** arrays (using {obj}`tensorflow <tf.Tensor>` and the [`phasespace`](https://phasespace.readthedocs.io) package as a back-end). We also need to construct a {class}`.RealNumberGenerator` that can generate random numbers. {class}`.TFUniformRealNumberGenerator` is the natural choice here.

As opposed to the main [](compwa-step-2) of the main usage example page, we will generate a **deterministic** data sample. This can be done by feeding a {class}`.RealNumberGenerator` with a specific {attr}`~.RealNumberGenerator.seed` and giving that generator to the {meth}`.TFPhaseSpaceGenerator.generate` method:

In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [ ]:
from tensorwaves.data import TFPhaseSpaceGenerator, TFUniformRealNumberGenerator

rng = TFUniformRealNumberGenerator(seed=0)
phsp_generator = TFPhaseSpaceGenerator(
    initial_state_mass=reaction.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in reaction.final_state.items()},
)
phsp_momenta = phsp_generator.generate(100_000, rng)

In [ ]:
import numpy as np
import pandas as pd

pd.DataFrame({
    (k, label): np.transpose(v)[i]
    for k, v in phsp_momenta.items()
    for i, label in enumerate(["E", "px", "py", "pz"])
})

The resulting phase space sample is a {obj}`dict` of final state IDs to an {obj}`~numpy.array` of four-momenta. In the last step, we converted this sample in such a way that it is rendered as an understandable {class}`pandas.DataFrame`.

:::{admonition} Four-momentum arrays
{doc}`Kinematic expressions <ampform:usage/kinematics>` from AmpForm that involve four-momenta should be formatted as $\left(E, p_x, p_y, p_z\right)$. In addition, the shape of input arrays should be `(n, 4)` with `n` the number of events.
:::

:::{warning}
When using the helicity formalism, the sum of the four-momenta should be in the rest frame, that is $\sum_i p_i = \left(m_A, 0, 0, 0\right)$ with $m_A$ the mass of the decaying particle&nbsp;$A$. This is because the helicity formalisms boosts through the decay chain starting from particle&nbsp;$A$. **Take care to boost your experimental data into the rest frame**, optionally following the {doc}`kinematics classes provided by AmpForm <ampform:usage/kinematics>`.
:::

In [ ]:
import numpy as np

p = np.array(list(phsp_momenta.values()))
p.shape

In [ ]:
p.sum(axis=0).round(decimals=14)

(compwa-step-2.2)=
### 2.2 Generate intensity-based sample

'Data samples' are more complicated than phase space samples in that they represent the intensity profile resulting from a reaction. You therefore need a {class}`.Function` object that expresses an intensity distribution as well as a phase space over which to generate that distribution. We call such a data sample an **intensity-based sample**.

An intensity-based sample is generated over a phase space sample using the {class}`.IntensityDistributionGenerator`. Its usage is similar to {class}`.TFPhaseSpaceGenerator`, but now you have to provide a {obj}`.Function` as well as a {obj}`.DataTransformer` that is used to transform the four-momentum phase space sample to a data sample that can be understood by the {obj}`.Function`.

Now, recall that in [](compwa-step-1), we used the helicity formalism to mathematically express the reaction in terms of an amplitude model. TensorWaves needs to convert this {obj}`~ampform.helicity.HelicityModel` to a {obj}`.Function` object that can perform fast computations. This can be done with {func}`.create_parametrized_function`:

In [ ]:
from tensorwaves.function.sympy import create_parametrized_function

unfolded_expression = model.expression.doit()
intensity_func = create_parametrized_function(
    expression=unfolded_expression,
    parameters=model.parameter_defaults,
    backend="numpy",
)

:::{tip}

If {func}`.create_parametrized_function` takes a long time, have a look at {doc}`usage/faster-lambdify`.

:::

:::{seealso}

{ref}`usage/basics:Hit & miss`

:::

A problem is that {class}`.ParametrizedBackendFunction` takes a {obj}`.DataSample` with kinematic variables for the helicity formalism as input, not a set of four-momenta. We therefore need to construct a {class}`.DataTransformer` to transform these four-momenta to function variables. In this case, we work with the helicity formalism, so we construct a {class}`.SympyDataTransformer`. This **numerical** data transformer is created from symbolic expressions, in this case, from expressions that relate four-momenta to helicity angles and invariant masses:

In [ ]:
Math(aslatex(model.kinematic_variables))

In [ ]:
from tensorwaves.data import SympyDataTransformer

helicity_transformer = SympyDataTransformer.from_sympy(
    model.kinematic_variables, backend="jax"
)

:::{seealso}

{ref}`usage:Generate and transform data`

:::

That's it, now we have enough info to create an intensity-based data sample. Notice how the structure of the output data is the same as the [phase-space sample we generated previously](compwa-step-2.1):

In [ ]:
from tensorwaves.data import (
    IntensityDistributionGenerator,
    TFWeightedPhaseSpaceGenerator,
)

weighted_phsp_generator = TFWeightedPhaseSpaceGenerator(
    initial_state_mass=reaction.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in reaction.final_state.items()},
)
data_generator = IntensityDistributionGenerator(
    domain_generator=weighted_phsp_generator,
    function=intensity_func,
    domain_transformer=helicity_transformer,
)
data_momenta = data_generator.generate(10_000, rng)
pd.DataFrame({
    (k, label): np.transpose(v)[i]
    for k, v in data_momenta.items()
    for i, label in enumerate(["E", "px", "py", "pz"])
})

As before, we use a {class}`.RealNumberGenerator` with a specific {attr}`~.RealNumberGenerator.seed` to ensure we get a **deterministic** data sample.

:::{note}

Instead of constructing a {class}`.TFWeightedPhaseSpaceGenerator`, it's also possible to just reuse the **unweighted** {class}`.TFPhaseSpaceGenerator` that we constructed previously. This is, however, a bit slower, because the {class}`.TFPhaseSpaceGenerator` also uses a hit-and-miss strategy.

:::

(compwa-step-2.3)=
### 2.3 Visualize kinematic variables

We now have a phase space sample and an intensity-based sample. Their data structure isn't the most informative though: it's just a collection of four-momentum tuples. But we can again use the {class}`.SympyDataTransformer` to convert these four-momenta to (in the case of the helicity formalism) invariant masses and helicity angles:

In [ ]:
phsp = helicity_transformer(phsp_momenta)
data = helicity_transformer(data_momenta)
list(data)

::::{note}
Check the remark about four-momentum format and rest frame of the decaying particle [here](compwa-step-2.1).
::::

The {obj}`.DataSample` is a mapping of kinematic variables names to a 1-dimensional array of values. The numbers you see here are final state IDs as defined in the {class}`~ampform.helicity.HelicityModel` member of the {class}`~ampform.helicity.HelicityModel`:

In [ ]:
for state_id, particle in reaction.final_state.items():
    print(f"ID {state_id}:", particle.name)

````{admonition} Available kinematic variables
---
class: dropdown
---
By default, {mod}`tensorwaves` only generates invariant masses of the {class}`Topologies <qrules.topology.Topology>` that are of relevance to the decay problem. In this case, we only have resonances $f_0 \to \pi^0\pi^0$. If you are interested in more invariant mass combinations, you can do so with the method {meth}`~ampform.kinematics.HelicityAdapter.register_topology`.
````

The {obj}`.DataSample` can easily be converted to a {class}`pandas.DataFrame`:

In [ ]:
import pandas as pd

data_frame = pd.DataFrame(data)
phsp_frame = pd.DataFrame(phsp)
data_frame

This also means that we can use all kinds of fancy plotting functionality of for instance {mod}`matplotlib.pyplot` to see what's going on. Here's an example:

In [ ]:
%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt

resonances = sorted(
    reaction.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [plt.cm.rainbow(x) for x in evenly_spaced_interval]
fig, ax = plt.subplots(figsize=(9, 4))
ax.hist(
    np.real(data_frame["m_12"]),
    bins=100,
    alpha=0.5,
    density=True,
)
ax.set_xlabel("$m$ [GeV]")
for p, color in zip(resonances, colors):
    ax.axvline(x=p.mass, linestyle="dotted", label=p.name, color=color)
ax.legend()
plt.show()

:::{seealso}

[](#fit-fractions)

:::

### 2.4 Export data sets

To export the generated data samples, simply {func}`pickle.dump` them as follows:

In [ ]:
import pickle

with open("data.pickle", "wb") as stream:
    pickle.dump(data, stream)
with open("phsp.pickle", "wb") as stream:
    pickle.dump(phsp, stream)

In the [next step](compwa-step-3), we illustrate how to {meth}`~.Minuit2.optimize` the intensity model to these data samples.

(compwa-step-3)=
## Step 3: Perform fit

As explained in the [previous step](compwa-step-2), a {class}`.ParametrizedFunction` can compute a list of intensities (real numbers) for an input {obj}`.DataSample`. At this stage, we want to optimize the parameters of this {class}`.ParametrizedFunction`, so that it matches the distribution of our data sample. This is what we call 'fitting'.

In [ ]:
reaction = qrules.io.load("transitions.json")
with open("helicity_model.pickle", "rb") as stream:
    model: HelicityModel = pickle.load(stream)
with open("data.pickle", "rb") as stream:
    data = pickle.load(stream)
with open("phsp.pickle", "rb") as stream:
    phsp = pickle.load(stream)

(compwa-step-3.1)=
### 3.1 Prepare parametrized function

In principle, we can use the same {class}`.ParametrizedFunction` as the one that we created in [](compwa-step-2.2). However, when fitting such a function to a data distribution, an {class}`.Optimizer` will evaluate this function numerous times, so it is smart to apply some optimizations to the underlying expression tree before-hand.

:::{tip}

The sections below illustrate some tricks for how to simplify the expression tree underneath a {class}`.ParametrizedFunction`. **Most of this can also be achieved with {func}`.create_cached_function`**, which is illustrated in {doc}`/usage/caching` and under [](compwa-create_cached_function). But note that it is still smart to [cast complex-valued data](#cast-complex-valued-data).

:::

#### Determine free parameters

It often happens that not all parameters in a {class}`.ParametrizedFunction` have to be optimized. These parameters are called **fixed parameters**, while the ones that will be optimized are called **free parameters**. In the fit example here, we will optimize the following free parameters, starting with a certain initial value.

In [ ]:
initial_parameters = {
    R"C_{J/\psi(1S) \to {f_{0}(1500)}_{0} \gamma_{+1}; f_{0}(1500) \to \pi^{0}_{0} \pi^{0}_{0}}": (
        1.0 + 0.0j
    ),
    "m_{f_{0}(500)}": 0.4,
    "m_{f_{0}(980)}": 0.88,
    "m_{f_{0}(1370)}": 1.22,
    "m_{f_{0}(1500)}": 1.45,
    "m_{f_{0}(1710)}": 1.83,
    R"\Gamma_{f_{0}(500)}": 0.3,
    R"\Gamma_{f_{0}(980)}": 0.1,
    R"\Gamma_{f_{0}(1710)}": 0.3,
}

In [ ]:
parameter_names = {symbol.name for symbol in model.parameter_defaults}
not_in_model = {
    par_name for par_name in initial_parameters if par_name not in parameter_names
}
if len(not_in_model) != 0:
    msg = f"Parameters {', '.join(sorted(not_in_model))} do not exist in model"
    raise ValueError(msg)

To make the fit more interesting, we give these initial values a small offset compared to the suggested {attr}`~ampform.helicity.HelicityModel.parameter_defaults`―after all, we are fitting to a data sample that was generated with this very same {class}`.ParametrizedFunction`.

:::{admonition} Complex-valued parameters

If initial parameter values are {obj}`complex`, the parameter is split into a real and an imaginary part during the fit. See also [](#covariance-matrix).

:::

#### Collapsing constant sub-trees

Having decided which parameters are to be optimized, we can apply some additional optimizations to the amplitude model expression, so that the computations during the fit are faster. The first of these optimizations is to substitute the parameters that remain fixed with their suggested parameter values. Note how this decreases the number of operations (nodes) in the expression tree:

In [ ]:
import sympy as sp

sp.count_ops(unfolded_expression)

In [ ]:
free_parameters = {p for p in model.parameter_defaults if p.name in initial_parameters}
fixed_parameters = {
    p: v for p, v in model.parameter_defaults.items() if p not in free_parameters
}
optimized_expression = unfolded_expression.subs(fixed_parameters)
sp.count_ops(optimized_expression)

In [ ]:
old = sp.count_ops(unfolded_expression)
new = sp.count_ops(optimized_expression)
assert old > new

Note that there are a few irrational numbers (square roots) in the expression as well, for example in this amplitude:

In [ ]:
optimized_expression.args[0].args[0].args[0].args[0]

When we substitute these values, the expression tree becomes even smaller:

In [ ]:
for node in sp.preorder_traversal(optimized_expression):
    if node.free_symbols:
        continue
    if isinstance(node, sp.Pow):
        optimized_expression = optimized_expression.xreplace({node: node.n()})
sp.count_ops(optimized_expression)

In [ ]:
old = sp.count_ops(unfolded_expression)
new = sp.count_ops(optimized_expression)
assert old > new

#### Substitute scalar masses

Since the final state particles $\gamma, \pi^0, \pi^0$ are **stable**, we can substitute their invariant masses $m_0, m_1, m_2$ with scalar values. SymPy will then further simply the expression, so that the computation is less complex.

In [ ]:
mass_substitutions = {
    sp.Symbol(f"m_{i}", nonnegative=True): particle.mass
    for i, particle in reaction.final_state.items()
}
optimized_expression = optimized_expression.subs(mass_substitutions)
sp.count_ops(optimized_expression)

In [ ]:
old = sp.count_ops(unfolded_expression)
new = sp.count_ops(optimized_expression)
assert old > new

#### Cast complex-valued data

Note that some of the computed kinematic variable arrays are complex-valued. This can be useful in come cases, but in this decay channel, all kinematic variable values lie on the real axis. The fit can therefore be further optimized by casting the data arrays to real values:

In [ ]:
from tensorwaves.interface import DataSample


def safe_downcast_to_real(data: DataSample) -> DataSample:
    # using isrealobj instead of real_if_close to keep same array backend
    return {k: v.real if np.isrealobj(v) else v for k, v in data.items()}

In [ ]:
data_real = safe_downcast_to_real(data)
phsp_real = safe_downcast_to_real(phsp)

#### Create computational function

Finally, we again use {func}`.create_parametrized_function` to create a {class}`.ParametrizedFunction` for this **optimized** expression:

In [ ]:
optimized_function = create_parametrized_function(
    optimized_expression, model.parameter_defaults, backend="jax"
)

Note that the intensities computed by the optimized function are indeed the same as the original intensity function that was created in [](compwa-step-2.2) and that it is much faster!

In [ ]:
# JIT-compile functions and test equality
np.testing.assert_array_almost_equal(
    optimized_function(data_real),
    intensity_func(data_real),
    decimal=13,
)

```{autolink-skip}
```

In [ ]:
%timeit -n1 intensity_func(data)

%timeit -n1 optimized_function(data_real)

The reason is that several sub-trees in the original expression tree have been collapsed, which results in fewer computations in the backend. Here's one of the sub-amplitudes, taken from the total expression:

##### Original expression tree

In [ ]:
dot = sp.dotprint(unfolded_expression.args[0].args[0].args[0].args[0])
graphviz.Source(dot)

##### Optimized expression tree

In [ ]:
dot = sp.dotprint(optimized_expression.args[0].args[0].args[0].args[0])
graphviz.Source(dot)

(compwa-create_cached_function)=
#### Simplified procedure: {func}`.create_cached_function`

As noted under [](compwa-step-3.1), most of what is described in this section can be achieved with the function {func}`.create_cached_function`. The idea is described on {doc}`/usage/caching`, but in this section, we show how this translates to amplitude analysis.

First, note that {func}`.create_cached_function` works with mappings and iterable of {class}`sympy.Symbol <sympy.core.symbol.Symbol>` and not with the {obj}`str` mappings that we defined in [](#determine-free-parameters). We can convert the parameter names back to {class}`~sympy.core.symbol.Symbol`s as follows:

In [ ]:
free_parameter_symbols = [
    symbol
    for symbol in model.parameter_defaults
    if symbol.name in set(initial_parameters)
]

In [ ]:
assert len(free_parameter_symbols) == len(initial_parameters)

This gives us all the information we need to create a cached function, convert the data samples to cached data samples and construct an {class}`.Estimator` with these transformed items (compare [](compwa-step-3.2)):

In [ ]:
from tensorwaves.estimator import UnbinnedNLL, create_cached_function

cached_function, cache_transformer = create_cached_function(
    unfolded_expression,
    parameters=model.parameter_defaults,
    free_parameters=free_parameter_symbols,
    backend="jax",
)
cached_data = cache_transformer(data_real)
cached_phsp = cache_transformer(phsp_real)
estimator_with_caching = UnbinnedNLL(
    cached_function,
    data=cached_data,
    phsp=cached_phsp,
    backend="jax",
)

Note that, just like in [](#create-computational-function), the computed intensities of both the original intensity function and the cached function are indeed the same:

In [ ]:
np.testing.assert_array_almost_equal(
    cached_function(cached_data),
    intensity_func(data_real),
    decimal=13,
)

```{autolink-skip}
```

In [ ]:
%timeit -n1 intensity_func(data_real)

%timeit -n1 cached_function(cached_data)

(compwa-step-3.2)=
### 3.2 Define estimator

To perform a fit, you need to define an {class}`.Estimator`. This is a measure for the discrepancy between the {class}`.ParametrizedFunction` and the data distribution to which you fit it. In PWA, we usually use an **unbinned negative log likelihood estimator** ({class}`.UnbinnedNLL`).

Generally, the {class}`.ParametrizedFunction` is not normalized with regards to the data sample, while a log likelihood estimator requires a normalized function. This is where the [phase space data](compwa-step-2.1) comes into play again: the {class}`.ParametrizedFunction` is evaluated over the phase space data, so that its output can be used as a normalization factor.

```{margin}
If you want to correct for the efficiency of the detector, you should use a *detector-reconstructed* phase space sample.
```

In [ ]:
from tensorwaves.estimator import UnbinnedNLL

estimator = UnbinnedNLL(
    optimized_function,
    data=data_real,
    phsp=phsp_real,
    backend="jax",
)

Note that the {class}`.UnbinnedNLL` can be expressed with different backends, because it uses statistical operations like `log` and `mean`. Here, we use {func}`jax <jax.jit>`, which turns out to be the fastest backend for this model.

(compwa-step-3.3)=
### 3.3 Optimize fit parameters

Starting the fit itself is quite simple: just create an {mod}`.optimizer` instance of your choice and call its {meth}`~.Optimizer.optimize` method to start the fitting process. The {meth}`~.Optimizer.optimize` method requires a mapping of parameter names to their initial values. **Only the parameters listed in the mapping are optimized.**

Let's have a look at our [first guess for the parameter values](#determine-free-parameters). Recall that a {class}`.ParametrizedFunction` object computes the intensity for a certain {obj}`.DataSample`. This can be seen nicely when we use these intensities as weights on the phase space sample and plot it together with the original data sample. Here, we look at the invariant mass distribution projection of the final states `1` and `2`, which, [as we saw before](compwa-step-2.3), is the final state particle pair $\pi^0\pi^0$.

Don't forget to use {meth}`~.ParametrizedFunction.update_parameters` first!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

resonances = sorted(
    reaction.get_intermediate_particles(),
    key=lambda p: p.mass,
)

evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [plt.cm.rainbow(x) for x in evenly_spaced_interval]


def indicate_masses(ax):
    ax.set_xlabel("$m$ [GeV]")
    for color, resonance in zip(colors, resonances):
        ax.axvline(
            x=resonance.mass,
            linestyle="dotted",
            label=resonance.name,
            color=color,
        )


def compare_model(
    variable_name,
    data,
    phsp,
    function,
    bins=100,
):
    intensities = function(phsp)
    _, ax = plt.subplots(figsize=(9, 4))
    data_projection = np.real(data[variable_name])
    ax = plt.gca()
    ax.hist(
        data_projection,
        bins=bins,
        alpha=0.5,
        label="data",
        density=True,
    )
    phsp_projection = np.real(phsp[variable_name])
    ax.hist(
        phsp_projection,
        weights=np.array(intensities),
        bins=bins,
        histtype="step",
        color="red",
        label="fit model",
        density=True,
    )
    indicate_masses(ax)
    ax.legend()

In [ ]:
original_parameters = optimized_function.parameters
optimized_function.update_parameters(initial_parameters)
compare_model("m_12", data_real, phsp_real, optimized_function)

Finally, we are ready to create an {class}`.Optimizer` that can {meth}`~.Minuit2.optimize` the parameters in the {class}`.ParametrizedFunction` (which is embedded in the {class}`.Estimator`). Here, we choose the {class}`.Minuit2` optimizer, which is the most common optimizer in high-energy physics (see also {ref}`usage/basics:Perform fit with different optimizers`). Since we constructed the {class}`.UnbinnedNLL` with {obj}`jax <jax.jit>`, can an optimize the model with analytic gradient over the {class}`.ParametrizedBackendFunction`:

:::{note}

The computation time depends on the complexity of the model, the number of data events, the size of the phase space sample, and the number of free parameters. This model is rather small and has but a few free parameters, so the optimization shouldn't take more than a minute.

:::

In [ ]:
from tensorwaves.optimizer import Minuit2
from tensorwaves.optimizer.callbacks import CSVSummary

minuit2 = Minuit2(
    callback=CSVSummary("fit_traceback.csv"),
    use_analytic_gradient=False,
)
fit_result = minuit2.optimize(estimator, initial_parameters)
fit_result

In [ ]:
assert fit_result.minimum_valid

:::{seealso}

See {ref}`usage/basics:Minuit2` for how to tweak the internal {class}`iminuit.Minuit` optimizer.

:::

As can be seen, the values of the optimized parameters in the {class}`.FitResult` are again comparable to the original parameter values.

In [ ]:
optimized_parameters = fit_result.parameter_values
for p in optimized_parameters:
    print(p)
    print(f"  initial:   {initial_parameters[p]:.3}")
    print(f"  optimized: {optimized_parameters[p]:.3}")
    print(f"  original:  {original_parameters[p]:.3}")

### 3.4 Export and import

In [](compwa-step-3.3), we initialized {obj}`.Minuit2` with a {class}`.Loadable` callback. Such callback classes offer the possibility to {meth}`~.Loadable.load_latest_parameters`, so you can pick up the optimize process in case it crashes or if you pause it. Loading the latest parameters goes as follows:

In [ ]:
latest_parameters = CSVSummary.load_latest_parameters("fit_traceback.csv")
latest_parameters

:::{seealso} 

{ref}`usage/basics:Callbacks` and [this example](./usage.ipynb#optimize-parameters) of a (custom) plotting callback.

:::

### 3.5 Analyze fit result

#### Plot optimized model

Using the same method as above, we renew the parameters of the {class}`.ParametrizedFunction` and plot it again over the phase space sample.

In [ ]:
optimized_function.update_parameters(fit_result.parameter_values)
compare_model("m_12", data_real, phsp_real, optimized_function)

#### Covariance matrix

Each of the {mod}`.optimizer`s offer more specific information about the fit result. This information can be accessed with {attr}`.FitResult.specifics`. A common example would be to get the {attr}`~iminuit.Minuit.covariance` matrix and {attr}`~iminuit.util.Matrix.correlation` matrix:

In [ ]:
covariance_matrix = fit_result.specifics.covariance
covariance_matrix.correlation()

:::{note}

Optimizers can only work with real numbers. For this reason, {obj}`complex`-valued parameters are split into a real and an imaginary part. This becomes apparent if we look at the covariance matrix.

:::

#### AIC and BIC

The [AIC](https://en.wikipedia.org/wiki/Akaike_information_criterion#Definition) and [BIC](https://en.wikipedia.org/wiki/Bayesian_information_criterion#Definition) give us another quantification of the quality of the fit. Here's how to compute them from this {obj}`.FitResult`:

In [ ]:
n_real_par = fit_result.count_number_of_parameters(complex_twice=True)
n_events = len(next(iter(data.values())))
log_likelihood = -fit_result.estimator_value

aic = 2 * n_real_par - 2 * log_likelihood
bic = n_real_par * np.log(n_events) - 2 * log_likelihood

In [ ]:
print("AIC:", aic)
print("BIC:", bic)

#### Fit fractions

As we have seen [when formulating the amplitude model](#build-amplitude-model), the helicity model is built up of an incoherent sum of real-valued intensities (one for each spin combination), which are each built up of a coherent sum of complex-valued amplitudes (one for each resonance). If we want to compute what each of these amplitudes contribute to the main intensity, we should use the optimized intensity and set coefficients or helicity couplings of amplitudes that were are not interested in to zero.

Here's an example function that can do this. Using regular expressions, we set all coefficients in the intensity function to zero if they do not contain a certain resonance $\LaTeX$ name:

In [ ]:
import re

from tensorwaves.interface import ParametrizedFunction


def compute_sub_intensity(
    func: ParametrizedFunction,
    input_data: DataSample,
    resonances: list[str],
):
    original_parameters = dict(func.parameters)
    negative_lookahead = f"(?!{'|'.join(map(re.escape, resonances))})"
    # https://regex101.com/r/WrgGyD/1
    pattern = rf"^(\\mathcal{{H}}|C_)({negative_lookahead}.)*$"
    set_parameters_to_zero(func, pattern)
    array = func(input_data)
    func.update_parameters(original_parameters)
    return array


def set_parameters_to_zero(func: ParametrizedFunction, name_pattern: str) -> None:
    new_parameters = dict(func.parameters)
    for par_name in func.parameters:
        if re.match(name_pattern, par_name) is not None:
            new_parameters[par_name] = 0
    func.update_parameters(new_parameters)

In [ ]:
all_resonances_latex = [p.latex for p in resonances]
np.testing.assert_array_equal(
    compute_sub_intensity(intensity_func, phsp, all_resonances_latex),
    intensity_func(phsp),
)
del all_resonances_latex

These functions can be used to compute and visualize the sub-intensity distributions over the phase space.

In [ ]:
total_intensities = intensity_func(phsp)
sub_intensities = {
    p: compute_sub_intensity(intensity_func, phsp, resonances=[p.latex])
    for p in resonances
}

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.set_xlim(0.25, 2.5)
ax.set_xlabel(R"$m_{\pi^0\pi^0}$ [GeV]")
ax.set_yticks([])

bins = 150
phsp_projection = phsp["m_12"].real
ax.hist(
    phsp_projection,
    weights=total_intensities,
    bins=bins,
    color="red",
    histtype="step",
    label="full intensity",
)
ax.hist(
    len(sub_intensities) * [phsp_projection],
    weights=list(sub_intensities.values()),
    bins=bins,
    alpha=0.6,
    label=[Rf"${p.latex}$" for p in sub_intensities],
    stacked=True,
)

fig.legend()
plt.tight_layout()
plt.show()

We can also use these functions to compute the decay rates for each resonance. Notice how the sum of the decay rates does not add up to a 100%. This is because of the strong constructive interference between the resonances in this model.

In [ ]:
total_intensity = intensity_func(phsp).sum()
fit_fractions = {
    resonance.name: f"{sub_intensity.sum() / total_intensity:.1%}"
    for resonance, sub_intensity in sub_intensities.items()
}
fit_fractions

## Advanced examples

```{toctree}
---
maxdepth: 2
---
amplitude-analysis/analytic-continuation
```